In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained =  './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.01
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style2/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    

    teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.010000
Train | 20/20 | Loss:1.0451 | MainLoss:1.0001 | Alpha:0.0235 | SPLoss:0.0635 | CLSLoss:1.8507 | top1:53.2368 | AUROC:0.0000
Test | 39/20 | Loss:0.7060 | MainLoss:0.7060 | SPLoss:0.0941 | CLSLoss:1.8230 | top1:52.7821 | AUROC:0.5435
Test | 161/20 | Loss:0.0646 | MainLoss:0.0646 | SPLoss:0.0941 | CLSLoss:1.8230 | top1:99.9439 | AUROC:1.0000

Epoch: [2 | 1000] LR: 0.013000
Train | 20/20 | Loss:1.2491 | MainLoss:0.6936 | Alpha:0.3319 | SPLoss:0.0006 | CLSLoss:1.6731 | top1:54.3158 | AUROC:0.0000
Test | 39/20 | Loss:0.6924 | MainLoss:0.6924 | SPLoss:0.0015 | CLSLoss:1.5156 | top1:53.5256 | AUROC:0.5569
Test | 161/20 | Loss:0.1180 | MainLoss:0.1180 | SPLoss:0.0015 | CLSLoss:1.5156 | top1:99.9533 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.016000
Train | 20/20 | Loss:1.1448 | MainLoss:0.6897 | Alpha:0.3337 | SPLoss:0.0003 | CLSLoss:1.3633 | top1:54.6316 | AUROC:0.0000
Test | 39/20 | Loss:0.6870 | MainLoss:0.6870 | SPLoss:0.0008 | CLSLoss:1.2065 | top1:54.6282 | AUR

Train | 20/20 | Loss:0.2755 | MainLoss:0.2268 | Alpha:0.4488 | SPLoss:0.0182 | CLSLoss:0.0903 | top1:91.9737 | AUROC:0.0000
Test | 39/20 | Loss:0.2588 | MainLoss:0.2588 | SPLoss:0.0307 | CLSLoss:0.0886 | top1:89.9359 | AUROC:0.9636
Test | 161/20 | Loss:0.3476 | MainLoss:0.3476 | SPLoss:0.0307 | CLSLoss:0.0886 | top1:85.5857 | AUROC:0.9958

Epoch: [24 | 1000] LR: 0.039986
Train | 20/20 | Loss:0.2486 | MainLoss:0.1999 | Alpha:0.4541 | SPLoss:0.0171 | CLSLoss:0.0899 | top1:93.0263 | AUROC:0.0000
Test | 39/20 | Loss:0.2401 | MainLoss:0.2401 | SPLoss:0.0301 | CLSLoss:0.0907 | top1:91.0641 | AUROC:0.9687
Test | 161/20 | Loss:0.4060 | MainLoss:0.4060 | SPLoss:0.0301 | CLSLoss:0.0907 | top1:83.4143 | AUROC:0.9930

Epoch: [25 | 1000] LR: 0.039983
Train | 20/20 | Loss:0.2329 | MainLoss:0.1847 | Alpha:0.4596 | SPLoss:0.0152 | CLSLoss:0.0898 | top1:93.7368 | AUROC:0.0000
Test | 39/20 | Loss:0.2392 | MainLoss:0.2392 | SPLoss:0.0287 | CLSLoss:0.0890 | top1:90.7308 | AUROC:0.9716
Test | 161/20 | Loss

Train | 20/20 | Loss:0.1068 | MainLoss:0.0667 | Alpha:0.4863 | SPLoss:0.0127 | CLSLoss:0.0700 | top1:98.2368 | AUROC:0.0000
Test | 39/20 | Loss:0.1789 | MainLoss:0.1789 | SPLoss:0.0259 | CLSLoss:0.0655 | top1:93.8846 | AUROC:0.9893
Test | 161/20 | Loss:0.8885 | MainLoss:0.8885 | SPLoss:0.0259 | CLSLoss:0.0655 | top1:71.7570 | AUROC:0.9453

Epoch: [46 | 1000] LR: 0.039886
Train | 20/20 | Loss:0.1284 | MainLoss:0.0922 | Alpha:0.4795 | SPLoss:0.0129 | CLSLoss:0.0625 | top1:97.4211 | AUROC:0.0000
Test | 39/20 | Loss:0.1659 | MainLoss:0.1659 | SPLoss:0.0262 | CLSLoss:0.0634 | top1:94.4103 | AUROC:0.9898
Test | 161/20 | Loss:0.9389 | MainLoss:0.9389 | SPLoss:0.0262 | CLSLoss:0.0634 | top1:70.4922 | AUROC:0.9471

Epoch: [47 | 1000] LR: 0.039879
Train | 20/20 | Loss:0.1179 | MainLoss:0.0813 | Alpha:0.4833 | SPLoss:0.0128 | CLSLoss:0.0627 | top1:97.7632 | AUROC:0.0000
Test | 39/20 | Loss:0.1543 | MainLoss:0.1543 | SPLoss:0.0190 | CLSLoss:0.0642 | top1:94.9615 | AUROC:0.9910
Test | 161/20 | Loss

Train | 20/20 | Loss:0.0840 | MainLoss:0.0532 | Alpha:0.4899 | SPLoss:0.0001 | CLSLoss:0.0629 | top1:98.7895 | AUROC:0.0000
Test | 39/20 | Loss:0.1341 | MainLoss:0.1341 | SPLoss:0.0002 | CLSLoss:0.0627 | top1:95.6282 | AUROC:0.9922
Test | 161/20 | Loss:1.1661 | MainLoss:1.1661 | SPLoss:0.0002 | CLSLoss:0.0627 | top1:66.2118 | AUROC:0.9375

Epoch: [68 | 1000] LR: 0.003969
Train | 20/20 | Loss:0.0825 | MainLoss:0.0517 | Alpha:0.4895 | SPLoss:0.0001 | CLSLoss:0.0627 | top1:98.6316 | AUROC:0.0000
Test | 39/20 | Loss:0.1344 | MainLoss:0.1344 | SPLoss:0.0002 | CLSLoss:0.0627 | top1:95.6026 | AUROC:0.9925
Test | 161/20 | Loss:1.1690 | MainLoss:1.1690 | SPLoss:0.0002 | CLSLoss:0.0627 | top1:66.2461 | AUROC:0.9370

Epoch: [69 | 1000] LR: 0.003968
Train | 20/20 | Loss:0.0874 | MainLoss:0.0568 | Alpha:0.4886 | SPLoss:0.0002 | CLSLoss:0.0624 | top1:98.5263 | AUROC:0.0000
Test | 39/20 | Loss:0.1332 | MainLoss:0.1332 | SPLoss:0.0003 | CLSLoss:0.0623 | top1:95.6923 | AUROC:0.9925
Test | 161/20 | Loss

Train | 20/20 | Loss:0.0697 | MainLoss:0.0398 | Alpha:0.4925 | SPLoss:0.0001 | CLSLoss:0.0607 | top1:99.2368 | AUROC:0.0000
Test | 39/20 | Loss:0.1317 | MainLoss:0.1317 | SPLoss:0.0001 | CLSLoss:0.0609 | top1:95.8462 | AUROC:0.9928
Test | 161/20 | Loss:1.1983 | MainLoss:1.1983 | SPLoss:0.0001 | CLSLoss:0.0609 | top1:66.1745 | AUROC:0.9360

Epoch: [90 | 1000] LR: 0.003940
Train | 20/20 | Loss:0.0793 | MainLoss:0.0495 | Alpha:0.4915 | SPLoss:0.0001 | CLSLoss:0.0605 | top1:98.9474 | AUROC:0.0000
Test | 39/20 | Loss:0.1358 | MainLoss:0.1358 | SPLoss:0.0002 | CLSLoss:0.0604 | top1:95.7564 | AUROC:0.9928
Test | 161/20 | Loss:1.1457 | MainLoss:1.1457 | SPLoss:0.0002 | CLSLoss:0.0604 | top1:67.2087 | AUROC:0.9401

Epoch: [91 | 1000] LR: 0.003939
Train | 20/20 | Loss:0.0750 | MainLoss:0.0453 | Alpha:0.4915 | SPLoss:0.0001 | CLSLoss:0.0604 | top1:98.8947 | AUROC:0.0000
Test | 39/20 | Loss:0.1328 | MainLoss:0.1328 | SPLoss:0.0002 | CLSLoss:0.0603 | top1:95.8077 | AUROC:0.9926
Test | 161/20 | Loss

Train | 20/20 | Loss:0.0679 | MainLoss:0.0385 | Alpha:0.4922 | SPLoss:0.0002 | CLSLoss:0.0596 | top1:99.2895 | AUROC:0.0000
Test | 39/20 | Loss:0.1306 | MainLoss:0.1306 | SPLoss:0.0003 | CLSLoss:0.0598 | top1:95.9872 | AUROC:0.9932
Test | 161/20 | Loss:1.2522 | MainLoss:1.2522 | SPLoss:0.0003 | CLSLoss:0.0598 | top1:65.4143 | AUROC:0.9283

Epoch: [112 | 1000] LR: 0.003902
Train | 20/20 | Loss:0.0812 | MainLoss:0.0519 | Alpha:0.4901 | SPLoss:0.0001 | CLSLoss:0.0597 | top1:98.7105 | AUROC:0.0000
Test | 39/20 | Loss:0.1284 | MainLoss:0.1284 | SPLoss:0.0002 | CLSLoss:0.0591 | top1:96.0385 | AUROC:0.9933
Test | 161/20 | Loss:1.2516 | MainLoss:1.2516 | SPLoss:0.0002 | CLSLoss:0.0591 | top1:65.2118 | AUROC:0.9269

Epoch: [113 | 1000] LR: 0.003900
Train | 20/20 | Loss:0.0766 | MainLoss:0.0476 | Alpha:0.4904 | SPLoss:0.0002 | CLSLoss:0.0589 | top1:98.7105 | AUROC:0.0000
Test | 39/20 | Loss:0.1285 | MainLoss:0.1285 | SPLoss:0.0004 | CLSLoss:0.0589 | top1:96.0256 | AUROC:0.9933
Test | 161/20 | Lo

Train | 20/20 | Loss:0.0645 | MainLoss:0.0358 | Alpha:0.4932 | SPLoss:0.0001 | CLSLoss:0.0581 | top1:99.3158 | AUROC:0.0000
Test | 39/20 | Loss:0.1251 | MainLoss:0.1251 | SPLoss:0.0002 | CLSLoss:0.0583 | top1:96.1667 | AUROC:0.9937
Test | 161/20 | Loss:1.2843 | MainLoss:1.2843 | SPLoss:0.0002 | CLSLoss:0.0583 | top1:64.8380 | AUROC:0.9211

Epoch: [134 | 1000] LR: 0.003855
Train | 20/20 | Loss:0.0686 | MainLoss:0.0399 | Alpha:0.4916 | SPLoss:0.0001 | CLSLoss:0.0582 | top1:99.0000 | AUROC:0.0000
Test | 39/20 | Loss:0.1276 | MainLoss:0.1276 | SPLoss:0.0003 | CLSLoss:0.0583 | top1:95.9615 | AUROC:0.9935
Test | 161/20 | Loss:1.2523 | MainLoss:1.2523 | SPLoss:0.0003 | CLSLoss:0.0583 | top1:65.4393 | AUROC:0.9232

Epoch: [135 | 1000] LR: 0.003853
Train | 20/20 | Loss:0.0679 | MainLoss:0.0392 | Alpha:0.4927 | SPLoss:0.0001 | CLSLoss:0.0582 | top1:99.4474 | AUROC:0.0000
Test | 39/20 | Loss:0.1272 | MainLoss:0.1272 | SPLoss:0.0002 | CLSLoss:0.0584 | top1:96.0128 | AUROC:0.9935
Test | 161/20 | Lo

Train | 20/20 | Loss:0.0667 | MainLoss:0.0386 | Alpha:0.4926 | SPLoss:0.0001 | CLSLoss:0.0570 | top1:99.1316 | AUROC:0.0000
Test | 39/20 | Loss:0.1231 | MainLoss:0.1231 | SPLoss:0.0002 | CLSLoss:0.0571 | top1:96.1795 | AUROC:0.9939
Test | 161/20 | Loss:1.3333 | MainLoss:1.3333 | SPLoss:0.0002 | CLSLoss:0.0571 | top1:64.1215 | AUROC:0.9164

Epoch: [156 | 1000] LR: 0.003799
Train | 20/20 | Loss:0.0631 | MainLoss:0.0348 | Alpha:0.4933 | SPLoss:0.0001 | CLSLoss:0.0572 | top1:99.3158 | AUROC:0.0000
Test | 39/20 | Loss:0.1266 | MainLoss:0.1266 | SPLoss:0.0002 | CLSLoss:0.0572 | top1:96.0769 | AUROC:0.9940
Test | 161/20 | Loss:1.2994 | MainLoss:1.2994 | SPLoss:0.0002 | CLSLoss:0.0572 | top1:64.8847 | AUROC:0.9198

Epoch: [157 | 1000] LR: 0.003796
Train | 20/20 | Loss:0.0693 | MainLoss:0.0412 | Alpha:0.4922 | SPLoss:0.0001 | CLSLoss:0.0571 | top1:99.2368 | AUROC:0.0000
Test | 39/20 | Loss:0.1322 | MainLoss:0.1322 | SPLoss:0.0002 | CLSLoss:0.0569 | top1:95.8718 | AUROC:0.9940
Test | 161/20 | Lo